In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Email_Spam_Prediction\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'c:\\Users\\HP\\Email_Spam_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    target_column: str

In [6]:
from emailproject.constants import *
from emailproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.multinomialnb
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from emailproject import logger
from sklearn.naive_bayes import MultinomialNB
import joblib

In [14]:
from sklearn.calibration import LabelEncoder


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        le = LabelEncoder()
        train_y = le.fit_transform(train_y)
        test_y = le.transform(test_y)


        lr = MultinomialNB()
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-09 10:33:20,625: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-09 10:33:20,625: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-09 10:33:20,625: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-09 10:33:20,631: INFO: common: created directory at: artifacts]
[2024-06-09 10:33:20,632: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\HP\Email_Spam_Prediction\emproj\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\HP\Email_Spam_Prediction\emproj\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ValueError: could not convert string to float: "Subject: institutional investor journals profile update confirmation  thank you for updating your ii journals profile information . changes to your user id , password and e - mail address have been made . please allow 24 hours for any other changes you ' ve made to take effect in our system . here is the profile information we have for you :  account number : 12973228  first name : vince  last name : kaminski  company name : enron corp  position : managing director  department :  address 1 : 1400 smith st eb 1962  address 2 :  city : houston  state : tx  zip code : 77002 - 7327  country : usa  phone : ( 713 ) 853 - 3848  extension :  fax : ( 713 ) 646 - 2503  foreign phone :  foreign fax :  email : vkamins @ enron . com  user id : vkaminski  password : italia"